# Procesamiento Lenguaje Natural

## *Debate cultural en el Rock Nacional*

### Abstract

Existe una asociación entre Rock Nacional Argentino, entendido como fenómeno contracultural de identificación juvenil y la busca de una expresión popular que refleje la situación coyuntural del país. El objetivo de este trabajo consiste en analizar la estrecha relación existente entre la construcción discursiva que tematizan las principales líricas del Rock Argentino con los hitos históricos sucedidos en dicho país en el transcurso de los años 1970-2010. Para poder llevar a cabo la investigación, se utilizó como corpus de más de 1000 artistas y un total superior a 24.000 canciones, extraídas del sitio web (www.rock.com.ar). 
El foco principal del estudio consistió en evaluar los tópicos presentes en las canciones tanto como la impronta sentimental en las mismas, haciendo especial énfasis en los cambios en las temáticas de interés a través de las décadas.

### Librerias

In [ ]:
# Librerias basicas
import numpy as np 
import pandas as pd
from pandas.core.common import flatten
import os
import random
import string
import math
from tqdm import tqdm
from collections import Counter
from operator import itemgetter
from datetime import datetime

#Librerias para scrapping
import requests
import bs4
import re
from bs4 import BeautifulSoup,SoupStrainer

# Librerias para NLP
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
from nltk import stem
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Plotting tools
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from wordcloud import WordCloud
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Varias
import warnings
from IPython.display import Markdown, display
warnings.simplefilter("ignore")

### Definición de funciones

#### 1. Función que busca el path de las encliclopedias

In [ ]:
#Buscamos las enciclopedias
def get_enciclopedias(URL):
    r = requests.get(URL)
    html = r.content.decode('utf-8')
    bhtml = bs4.BeautifulSoup(html)
    enciclopedia = []
    for link in bhtml.find_all('a', attrs={'href': re.compile("^/enciclopedia")}):
        enciclopedia.append(link.get('href'))
    del enciclopedia[0]
    return enciclopedia

#### 2. Función que busca el path de los artistas

In [ ]:
#Buscamos los artistas dentro de cada enciclopedia
def get_artistas(URL):
    r = requests.get(URL)
    html = r.content.decode('utf-8')
    bhtml = bs4.BeautifulSoup(html)
    artista = []
    for link in bhtml.find_all('a', attrs={'href': re.compile("^/artista")}):
            artista.append(link.get('href'))
    return artista

#### 3. Función que scrapea la metadata y el path de las canciones

In [ ]:
def get_bandas(artista):
    # Variables
    nombre_artista = [] ; path_discos = [] ; fecha_edicion = [] ; canciones = [[],[],[],[]] 
    # Get de url
    disc_url = 'https://www.rock.com.ar' + artista + '/discos'
    r = requests.get(disc_url)
    html = r.content.decode('utf-8') ; bhtml = bs4.BeautifulSoup(html)
    # Links a los Discos
    for link in bhtml.find_all('a', attrs={'href': re.compile("^" + artista + "/discos")}):
        path_discos.append(link.get('href'))
    path_discos = path_discos[1:-1]
    path_discos = np.unique(path_discos)             
    # Discos-Fecha-Canciones de las canciones en cada Disco
    for i in range(len(path_discos)):
        # Get de url
        disc_url = 'https://www.rock.com.ar' + path_discos[i]
        r = requests.get(disc_url)
        html = r.content.decode('utf-8') ; bhtml = bs4.BeautifulSoup(html)
        #Artista
        nombre_artista = bhtml.find('h1').text
        #Año publicacion
        fecha_edicion= bhtml.find_all(re.compile('^p'))[1].text
        fecha_edicion = fecha_edicion.split(':')[1]
        if len(fecha_edicion.split('/')) != 1:
            fecha_edicion = fecha_edicion.split('/')[2]
        #Disco y canciones
        for link in bhtml.find_all('a', attrs={'href': re.compile("^"+'/'.join(path_discos[i].split('/')[0:3]) + '/letras')}):
            if len(link.get('href').split('/'))>4:
                canciones[0].append(nombre_artista)                
                canciones[1].append(path_discos[i])
                canciones[2].append(fecha_edicion)
                canciones[3].append(link.get('href'))
    return canciones

#### 4. Función que busca las letras de las canciones

In [ ]:
#Te devuelve el corpus de las liricas de las canciones
def get_liricas(URL):
    r = requests.get(URL)
    html = r.content.decode('utf-8')
    bhtml = bs4.BeautifulSoup(html)
    letras = bhtml.find('div', attrs={'class':'post-content-text'})
    letras = letras.get_text()
    liricas = re.split('\n\n',letras)[3]
    return liricas

#### 5. Función que limpa el texto de las canciones

In [ ]:
def limpiar_cancion(cancion):
    cancion = str(cancion)
    cancion = cancion.replace(".", " ")                             # Reemplazamos puntos por espacios
    cancion = re.sub("\d+", " ", cancion)                           # Eliminamos de caracteres numericos.
    cancion = cancion.replace('\n', ' ')                            # Eliminamos salto de parrafo.
    cancion = cancion.replace('\r', ' ')                            # Eliminamos de nuevas lineas.
    cancion = cancion.translate(str.maketrans("", "", punctuation)) # Eliminamos caracteres de puntuacion ( !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ).
    cancion = re.sub('([A-Z])', r' \1', cancion)                    # Espacio antes de mayusculas 
    cancion = cancion.lower()                                       # Convertimos a minuscula.
    cancion = re.sub("  +", " ", cancion)                           # Eliminamos mas de 2 espacios
    return cancion

#### 6. Función que normaliza el texto de las canciones

In [ ]:
nlp = spacy.load("es_core_news_sm")
def normalizar_cancion(cancion):
    doc = nlp(cancion)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop] # Crea una lista de tokens sacando stopwords
    tokens = [t for t in words if len(t) > 3] #Eliminamos tokens con menos de 3 letras
    lemmas = []
    for i in range(len(tokens)):
        doc = nlp(tokens[i])
        lemmas.append([w.lemma_ for w in doc])    
    return list(flatten(lemmas))

### Web scrapping

Escrapeamos las enciclopedias en busqueda de los artistas con sus discografía

In [ ]:
URL = 'https://www.rock.com.ar/'
enciclopedia = [] ; artista = [] ; banda_rock = []
enciclopedia = get_enciclopedias(URL)
for i in tqdm(range(len(enciclopedia)-1)):
    URL = 'https://www.rock.com.ar' + enciclopedia[i]
    artista = get_artistas(URL)
    seleccion = random.sample(artista,math.floor(len(artista)*1))
    for j in range(len(seleccion)):
        banda_rock.append(get_bandas(seleccion[j]))

Transformamos las listas a Dataframe

In [ ]:
df = pd.DataFrame(columns=['Artista','Path_disco','Fecha_edicion','Path_canciones'])
for n in range(len(banda_rock)):
    df_temp = pd.DataFrame(columns=['Artista','Path_disco','Fecha_edicion','Path_canciones'])
    df_temp = pd.DataFrame(banda_rock[n]).T
    df_temp.columns = ['Artista','Path_disco','Fecha_edicion','Path_canciones']
    frames = [df,df_temp]
    if len(df_temp)>0 : df = pd.concat(frames,ignore_index=True)

Definimos a que decada pertenece cada cancion y buscamos su lirica (letra).

In [ ]:
# Definicion de decada
trunc = lambda x: math.trunc(x/10) * 10
Decada = pd.to_numeric(df.Fecha_edicion).map(trunc)
df.insert(3, "Decada", Decada)
# Busqueda de lirica
for i in tqdm(range(len(df))):
    URL = 'https://www.rock.com.ar' + df.Path_canciones[i]
    df.loc[df['Path_canciones']==df.Path_canciones[i],'Liricas']=get_liricas(URL)

### Preprocesamiento

En esta sección realizaremos la limpieza y normalizacion del corpus de las canciones:

In [ ]:
# Limpieza de caracteres no deseados en el corpus
df.Liricas = df.Liricas.apply(limpiar_cancion)
# Normalizacion del corpus
df.Liricas = df.Liricas.apply(normalizar_cancion)
# Eliminacion de canciónes repetidas (presentes en mas de un disco)
df = df.sort_values("Fecha_edicion")
df = df.drop_duplicates(subset="Path_canciones", keep="first")
df = df.drop_duplicates(subset="Liricas", keep="first")

Exportamos el dataset a una base de datos para tener la información almacenada localmente.

In [ ]:
df.to_csv("BD_Rock_Nacional.csv",index=False)

### Topic Models & Sentiment analysis

In [ ]:
# Cargamos la base de datos
df = pd.read_csv("BD_Rock_Nacional.csv")
# Reconversión de las Liricas a Tokens (str por el csv)
tokens = df.Liricas
tokens = [n.strip('][').split(', ') for n in tokens]
tokens = [[x.replace("'", "") for x in i] for i in tokens]
df.insert(6,"Tokens",tokens)

#### Análisis exploratorio

In [ ]:
resumen_fecha = df.groupby(['Fecha_edicion']).agg(['nunique','count'])
fig,(ax1,ax3) = plt.subplots(1,2,figsize=(30,10))
fig.suptitle('Canciones y Artistas por año/decada', fontsize=35)
ax1.plot(resumen_fecha.index,resumen_fecha.iloc[:,0], color="red", marker="o")
ax1.set_xlabel("Fecha_edicion",fontsize=20)
ax1.set_ylabel("Artistas",color="red",fontsize=20)
plt.grid(True)
ax2 = ax1.twinx()
ax2.plot(resumen_fecha.index,resumen_fecha.iloc[:,5],color="blue",marker="o")
resumen_decada = df.groupby(['Decada']).agg(['nunique','count'])
ax3.plot(resumen_decada.index,resumen_decada.iloc[:,0], color="red", marker="o")
ax3.set_xlabel("Decada",fontsize=20)
plt.grid(True)
ax4 = ax3.twinx()
ax4.plot(resumen_decada.index,resumen_decada.iloc[:,5],color="blue",marker="o")
ax4.set_ylabel("Canciones",color="blue",fontsize=20)
plt.show()

Creamos un pareto para conocer los artistas más influyentes

In [ ]:
resumen_artista = df.groupby(['Artista']).count()
resumen_artista = resumen_artista.sort_values("Liricas",ascending=False)
resumen_artista["cumpercentage"] = resumen_artista["Liricas"].cumsum()/resumen_artista["Liricas"].sum()*100
pareto = len(resumen_artista.loc[resumen_artista["cumpercentage"]<=80]) / len(resumen_artista)
print("El",round(pareto*100),"% de los artistas compuso el 80% del total de las canciones de la base de datos")

Creamos los 40 principales artistas de la base de datos

In [ ]:
top = resumen_artista.iloc[0:40,:]
fig, ax = plt.subplots(figsize=(30,10))
fig.suptitle('Los 40 principales', fontsize=35)
ax.bar(top.index, top["Liricas"], color="C0")
plt.xticks(rotation=90,fontsize=15)
ax2 = ax.twinx()
ax2.plot(top.index, top["cumpercentage"], color="C1", marker="D", ms=7)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax.tick_params(axis="y", colors="C0")
ax2.tick_params(axis="y", colors="C1")
plt.show()

#### Wordcloud por decada

In [ ]:
def sacar_comilla(cancion):
    cancion = cancion.replace("'", "")      # Eliminamos comillas para mejor interpretacion
    return cancion
Words = pd.DataFrame(df.Tokens.astype(str).apply(sacar_comilla))
Words.insert(0,"Decada",df.Decada)

In [ ]:
canciones_70 = ''.join([str(n) for n in Words[Words['Decada']==1970]['Tokens']])
canciones_70_wc = WordCloud(background_color="white",colormap="Dark2",max_font_size=100,random_state=15,width=512, height=512).generate(canciones_70)
canciones_80 = ''.join([str(n) for n in Words[Words['Decada']==1980]['Tokens']])
canciones_80_wc = WordCloud(background_color="white",colormap="Dark2",max_font_size=100,random_state=15,width=512, height=512).generate(canciones_80)
canciones_90 = ''.join([str(n) for n in Words[Words['Decada']==1990]['Tokens']])
canciones_90_wc = WordCloud(background_color="white",colormap="Dark2",max_font_size=100,random_state=15,width=512, height=512).generate(canciones_90)
canciones_00 = ''.join([str(n) for n in Words[Words['Decada']==2000]['Tokens']])
canciones_00_wc = WordCloud(background_color="white",colormap="Dark2",max_font_size=100,random_state=15,width=512, height=512).generate(canciones_00)
fig, ax = plt.subplots(2,2, figsize = (20,20))
ax[0,0].imshow(canciones_70_wc)
ax[0,0].axis('off')
ax[0,0].set_title('Decada 1970s', fontsize = 30)
ax[0,1].imshow(canciones_80_wc)
ax[0,1].axis('off')
ax[0,1].set_title('Decada 1980s', fontsize = 30)
ax[1,0].imshow(canciones_90_wc)
ax[1,0].axis('off')
ax[1,0].set_title('Decada 1990s', fontsize = 30)
ax[1,1].imshow(canciones_00_wc)
ax[1,1].axis('off')
ax[1,1].set_title('Decada 2000', fontsize = 30)
plt.show()

Evolución de las palabras a través de los años

In [ ]:
Words70 = pd.DataFrame(canciones_70_wc.words_.items(), columns=['Palabra', '1970s']).iloc[0:10]
Words80 = pd.DataFrame(canciones_80_wc.words_.items(), columns=['Palabra', '1980s']).iloc[0:10]
Words90 = pd.DataFrame(canciones_90_wc.words_.items(), columns=['Palabra', '1990s']).iloc[0:10]
Words00 = pd.DataFrame(canciones_00_wc.words_.items(), columns=['Palabra', '2000s']).iloc[0:10]
Topwords10 = Words70.merge(Words80,on='Palabra',how='outer').merge(Words90,on='Palabra',how='outer').merge(Words00,on='Palabra',how='outer')
freq70 = pd.DataFrame(canciones_70_wc.words_.items(), columns=['Palabra', '1970s']).iloc[0:100]
freq80 = pd.DataFrame(canciones_80_wc.words_.items(), columns=['Palabra', '1980s']).iloc[0:100]
freq90 = pd.DataFrame(canciones_90_wc.words_.items(), columns=['Palabra', '1990s']).iloc[0:100]
freq00 = pd.DataFrame(canciones_00_wc.words_.items(), columns=['Palabra', '2000s']).iloc[0:100]
Topwords100 = freq70.merge(freq80,on='Palabra',how='outer').merge(freq90,on='Palabra',how='outer').merge(freq00,on='Palabra',how='outer')
Topwords10 = pd.DataFrame(Topwords10['Palabra'])
Topwords10 = Topwords10.merge(Topwords100 ,on='Palabra',how='left')
Topwords10 = Topwords10.set_index('Palabra')
Topwords10 = Topwords10.transpose()
Topwords10 = Topwords10.replace(np.nan, 0)
# Subset de palabras
Topwords = pd.DataFrame(Topwords10[["vivir","pensar","morir","esperar","mundo","vida","sentir","noche","decir","corazón"]])
plt.figure()
Topwords.plot(subplots=True, layout=(15, 5), figsize=(20, 30))
plt.show()

#### Topic Models (LDA)

##### Número óptimo de tópicos en función de la Coherencia

In [ ]:
#Funcion para evaluar el número óptimo de Tópicos según la Coherencia
def learn_lda_model(corpus, dictionary, texts,k):
    lda = LdaModel(corpus, 
               id2word=dictionary, 
               num_topics=k, 
               random_state=37, 
               iterations=100,
               passes=5,
               per_word_topics=False)
    cm = CoherenceModel(model=lda, corpus=corpus, texts=texts, coherence='c_v') 
    coherence = cm.get_coherence()
    print('{}: {}'.format(k, coherence))
    return k, coherence

In [ ]:
Liricas = df.Tokens
dictionary = corpora.Dictionary(Liricas)
corpus = [dictionary.doc2bow(text) for text in Liricas]

In [ ]:
lda_scores = [learn_lda_model(corpus, dictionary, Liricas,k) for k in tqdm(range(2, 15))]

In [ ]:
def plot_scores(scores, ax, ylabel):
    _x = [s[0] for s in scores]
    _y = [s[1] for s in scores]
    ax.plot(_x, _y, color='tab:blue')
    ax.set_xlabel('k')
    ax.set_ylabel(ylabel)
    ax.set_title('{} vs k'.format(ylabel))

fig, ax = plt.subplots(1, 1, figsize=(15, 5))
plot_scores(lda_scores, ax, 'LDA Coherence')
plt.axvline(x=5,color="red",linestyle="dashed")
plt.grid()
plt.tight_layout()

##### Topic Models ( k = 5 )

In [ ]:
N_TOPICS = 5 #max(lda_scores,key=itemgetter(1))[0]

# LDA
common_texts = Liricas
# Create a corpus from a list of texts
id2word = Dictionary(common_texts)
corpus = [id2word.doc2bow(text) for text in common_texts]

# # Train the model on the corpus.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=N_TOPICS, 
                                           random_state=100,
                                            update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)
pyLDAvis.display(lda_display)

Asignación del tópico correspondiente a cada canción, siendo este el de mayor probabilidad. 

In [ ]:
train_vecs = []
for i in range(len(Liricas)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(N_TOPICS)]
    train_vecs.append(topic_vec)
train_vecs = pd.DataFrame(train_vecs)
df['Topic'] = train_vecs.idxmax(axis=1, skipna=True)
df['Topic_proba'] = train_vecs.max(axis=1)
df[df['Topic'] == 0].nlargest(5,"Topic_proba")

#### Sentiment Analysis

##### Construcción Lexicon en Español

In [ ]:
xlsx = pd.read_excel("NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx")
xlsx = xlsx[['Spanish (es)','Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
xlsx.columns = ['Español','Positivo','Negativo','Enojo','Esperanza','Asco','Miedo','Alegria','Tristeza','Sorpresa','Confianza']

In [ ]:
columns_to_iterate = ['Positivo','Negativo','Enojo','Esperanza','Asco','Miedo','Alegria','Tristeza','Sorpresa','Confianza']
with open('lexicon_spanish_2020.txt', 'a') as f:
    for index, a_word in xlsx.iterrows():
        for a_column in columns_to_iterate:
            line = "%s\t%s\t%d\n" % (a_word['Español'], a_column, a_word[a_column])
            f.write(line)

##### Análisis de Emociones y Sentimientos

In [ ]:
# Importamos las herramientas a utilizar
from py_lex import Liwc, EmoLex
lexicon = EmoLex('lexicon_spanish_2020.txt')

In [ ]:
# Creamos las columnas con los sentimientos
df['Positivo'] = 0.0
df['Negativo'] = 0.0
df['Enojo'] = 0.0
df['Asco'] = 0.0
df['Miedo'] = 0.0
df['Tristeza'] = 0.0
df['Alegria'] = 0.0
df['Sorpresa'] = 0.0
df['Confianza'] = 0.0
df['Esperanza'] = 0.0

In [ ]:
# Asignamos el grado de probabilidad que posee sobre las sentimientos / emociones definidas
for index, _ in df.iterrows():
    try:
        summary = lexicon.summarize_doc(df["Tokens"][index])
        for key in summary.keys():
            df.at[index, key] = summary[key]
    except:
        continue

##### Asignación de Sentimientos / Emociones y Tópicos a las canciones
Definimos el sentimiento que representa cada canción (criterio de mayor probabilidad).

In [ ]:
df_final = df.iloc[:,0:9]
#Sentimientos
df_final["Sentimiento"]= df[['Positivo','Negativo']].idxmax(axis="columns", skipna=True)
df_final["Sentimiento_proba"]= df[['Positivo','Negativo']].max(axis=1)
#Emociones
df_final["Emocion"]= df[['Enojo','Asco','Miedo','Tristeza','Alegria','Sorpresa','Confianza','Esperanza']].idxmax(axis="columns", skipna=True)
df_final["Emocion_proba"]= df[['Enojo','Asco','Miedo','Tristeza','Alegria','Sorpresa','Confianza','Esperanza']].max(axis=1)
# Eliminamos los NAN
df_final = df_final.dropna()

Asignamos los nombres / etiquetas a los topicos encontrados

In [ ]:
df_final["Topic"] = df_final["Topic"].replace(0,"Esperanza")
df_final["Topic"] = df_final["Topic"].replace(1,"Ingles")
df_final["Topic"] = df_final["Topic"].replace(2,"Festividad")
df_final["Topic"] = df_final["Topic"].replace(3,"Romanticismo")
df_final["Topic"] = df_final["Topic"].replace(4,"Violencia")

### Resultados

In [ ]:
# Cargamos la base de datos con los topicos y sentimientos
df_final = pd.read_csv("Rock_Nacional_Final.csv")

#### Emociones
Negativas durante la decada 70-90s

In [ ]:
desde = 1970 ; hasta = 1990
df_final.loc[df_final["Decada"].isin(range(desde,hasta)) & df_final["Emocion"].isin(["Miedo","Enojo","Tristeza"])].pivot_table(index='Fecha_edicion',columns='Emocion',values='Liricas',aggfunc=len, fill_value=0).plot(figsize=(30,10))
# Hitos históricos de la Argentia
plt.axvline(1975,color="grey",linewidth=4,linestyle="dashed")           # Rodrigazo
plt.text(1975,0,"Rodrigazo",fontsize=20,color="black",rotation=90)
plt.axvline(1976,color="grey",linewidth=4,linestyle="dashed")           # Dictadura
plt.text(1976,0,"Dictadura",fontsize=20,color="black",rotation=90)
plt.axvline(1978,color="grey",linewidth=4,linestyle="dashed")           # Mundial
plt.text(1978,0,"Mundial",fontsize=20,color="black",rotation=90)
plt.axvline(1982,color="grey",linewidth=4,linestyle="dashed")           # Guerra de las Malvinas
plt.text(1982,0,"Guerra de las Malvinas",fontsize=20,color="black",rotation=90)       
plt.axvline(1983,color="grey",linewidth=4,linestyle="dashed")           # Vuelta de la democracia
plt.text(1983,0,"Vuelta de la Democracia",fontsize=20,color="black",rotation=90)      
plt.axvline(1989,color="grey",linewidth=4,linestyle="dashed")           # Hiperinflación
plt.text(1989,0,"Hiperinflación",fontsize=20,color="black",rotation=90)               
#plt.axvline(2001,color="grey",linewidth=4,linestyle="dashed")          # Crisis del 2001
#plt.text(2001,0,"Crisis 2001",fontsize=20,color="black",rotation=90)                  
plt.title("Emociones negativas desde %s" % desde + " hasta %s" % hasta,fontsize=35)
plt.xlabel("Fecha de edición", fontsize=20)
plt.ylabel("# Canciones", fontsize=20)
plt.xlim(desde,hasta)
plt.xticks(ticks=range(desde,hasta),fontsize=20,rotation=90)
plt.yticks(fontsize=20)
plt.legend(fontsize=25,loc="upper left")
plt.grid()
plt.show()

Positivas decada 70-90s

In [ ]:
desde = 1970 ; hasta = 1990
df_final.loc[df_final["Decada"].isin(range(desde,hasta)) & df_final["Emocion"].isin(['Alegria','Confianza','Esperanza'])].pivot_table(index='Fecha_edicion',columns='Emocion',values='Liricas',aggfunc=len, fill_value=0).plot(figsize=(30,10))
#Hitos históricos de la Argentia
plt.axvline(1975,color="grey",linewidth=4,linestyle="dashed")           # Rodrigazo
plt.text(1975,0,"Rodrigazo",fontsize=20,color="black",rotation=90)
plt.axvline(1976,color="grey",linewidth=4,linestyle="dashed")           # Dictadura
plt.text(1976,0,"Dictadura",fontsize=20,color="black",rotation=90)
plt.axvline(1978,color="grey",linewidth=4,linestyle="dashed")           # Mundial
plt.text(1978,0,"Mundial",fontsize=20,color="black",rotation=90)
plt.axvline(1982,color="grey",linewidth=4,linestyle="dashed")           # Guerra de las Malvinas
plt.text(1982,0,"Guerra de las Malvinas",fontsize=20,color="black",rotation=90)       
plt.axvline(1983,color="grey",linewidth=4,linestyle="dashed")           # Vuelta de la democracia
plt.text(1983,0,"Vuelta de la Democracia",fontsize=20,color="black",rotation=90)      
plt.axvline(1989,color="grey",linewidth=4,linestyle="dashed")           # Hiperinflación
plt.text(1989,0,"Hiperinflación",fontsize=20,color="black",rotation=90)               
#plt.axvline(2001,color="grey",linewidth=4,linestyle="dashed")          # Crisis del 2001
#plt.text(2001,0,"Crisis 2001",fontsize=20,color="black",rotation=90)                  
plt.title("Emociones postivas desde %s" % desde + " hasta %s" % hasta,fontsize=35)
plt.xlabel("Fecha de edición", fontsize=20)
plt.ylabel("# Canciones", fontsize=20)
plt.xlim(desde,hasta)
plt.xticks(ticks=range(desde,hasta),fontsize=20,rotation=90)
plt.yticks(fontsize=20)
plt.legend(fontsize=25,loc="upper left")
plt.grid()
plt.show()

#### Topicos 
Decada 1970-90s

In [ ]:
desde = 1970 ; hasta = 1990
df_final.loc[df_final["Decada"].isin(range(desde,hasta)) & df_final["Topic"].isin(["Esperanza","Festividad","Romanticismo","Violencia"])].pivot_table(index='Fecha_edicion',columns='Topic',values='Liricas',aggfunc=len, fill_value=0).plot(figsize=(30,10))
#Hitos históricos de la Argentia
plt.axvline(1975,color="grey",linewidth=4,linestyle="dashed")                 #Rodrigazo
plt.text(1975,0,"Rodrigazo",fontsize=20,color="black",rotation=90)
plt.axvline(1976,color="grey",linewidth=4,linestyle="dashed")                 #Dictadura
plt.text(1976,0,"Dictadura",fontsize=20,color="black",rotation=90)
plt.axvline(1978,color="grey",linewidth=4,linestyle="dashed")               #Mundial
plt.text(1978,0,"Mundial",fontsize=20,color="black",rotation=90)
plt.axvline(1982,color="grey",linewidth=4,linestyle="dashed")                 #Guerra de las Malvinas
plt.text(1982,0,"Guerra de las Malvinas",fontsize=20,color="black",rotation=90)       
plt.axvline(1983,color="grey",linewidth=4,linestyle="dashed")               #Vuelta de la democracia
plt.text(1983,0,"Vuelta de la Democracia",fontsize=20,color="black",rotation=90)      
plt.axvline(1989,color="grey",linewidth=4,linestyle="dashed")                 #Hiperinflación
plt.text(1989,0,"Hiperinflación",fontsize=20,color="black",rotation=90)               
#plt.axvline(2001,color="grey",linewidth=4,linestyle="dashed")                 #Crisis del 2001
#plt.text(2001,0,"Crisis 2001",fontsize=20,color="black",rotation=90)                  
plt.title("Topicos desde %s" % desde + " hasta %s" % hasta,fontsize=35)
plt.xlabel("Fecha de edición", fontsize=20)
plt.ylabel("# Canciones", fontsize=20)
plt.xlim(desde,hasta)
plt.xticks(ticks=range(desde,hasta),fontsize=20,rotation=90)
plt.yticks(fontsize=20)
plt.legend(fontsize=25,loc="upper left")
plt.grid()
plt.show()

#### Sentimientos
Decadas 1970-1990s

In [ ]:
desde = 1970 ; hasta = 1990
df_final.loc[df_final["Decada"].isin(range(desde,hasta)) & df_final["Sentimiento"].isin(['Positivo','Negativo'])].pivot_table(index='Fecha_edicion',columns='Sentimiento',values='Liricas',aggfunc=len, fill_value=0).plot(figsize=(30,10))
#Hitos históricos de la Argentia
plt.axvline(1975,color="grey",linewidth=4,linestyle="dashed")           # Rodrigazo
plt.text(1975,0,"Rodrigazo",fontsize=20,color="black",rotation=90)
plt.axvline(1976,color="grey",linewidth=4,linestyle="dashed")           # Dictadura
plt.text(1976,0,"Dictadura",fontsize=20,color="black",rotation=90)
plt.axvline(1978,color="grey",linewidth=4,linestyle="dashed")           # Mundial
plt.text(1978,0,"Mundial",fontsize=20,color="black",rotation=90)
plt.axvline(1982,color="grey",linewidth=4,linestyle="dashed")           # Guerra de las Malvinas
plt.text(1982,0,"Guerra de las Malvinas",fontsize=20,color="black",rotation=90)       
plt.axvline(1983,color="grey",linewidth=4,linestyle="dashed")           # Vuelta de la democracia
plt.text(1983,0,"Vuelta de la Democracia",fontsize=20,color="black",rotation=90)      
plt.axvline(1989,color="grey",linewidth=4,linestyle="dashed")           # Hiperinflación
plt.text(1989,0,"Hiperinflación",fontsize=20,color="black",rotation=90)               
#plt.axvline(2001,color="grey",linewidth=4,linestyle="dashed")          # Crisis del 2001
#plt.text(2001,0,"Crisis 2001",fontsize=20,color="black",rotation=90)                  
plt.title("Sentimientos desde %s" % desde + " hasta %s" % hasta,fontsize=35)
plt.xlabel("Fecha de edición", fontsize=20)
plt.ylabel("# Canciones", fontsize=20)
plt.xlim(desde,hasta)
plt.xticks(ticks=range(desde,hasta),fontsize=20,rotation=90)
plt.yticks(fontsize=20)
plt.legend(fontsize=25,loc="upper left")
plt.grid()
plt.show()